# Kapitel 11 - Teil 2


In [ ]:
import geopandas as gpd

Laden eines ESRI Shapefiles, andere Formate, z.B. GeoJSON werden auch unterstützt. Die vollständige Liste unterstützter Vektor-Formate erfolgt über das uns schon bekannte fiona Modul:

    import fiona
    fiona.supported_drivers

In [ ]:
import fiona
fiona.supported_drivers

### GeoDataFrame aus einem CSV erstellen

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('daten/cities5000.txt', sep="\t", header=None, low_memory=False)

In [ ]:
df.head()

Query: Nur bestimmte Spalten:

In [ ]:
df2 = df[[1,4,5]]
df2.head()

Spalten benennen

In [ ]:
df2.columns = ["name", "lat", "lng"]

In [ ]:
df2.head()

In [ ]:
len(df2)

In [ ]:
df2.shape

In [ ]:
df2.query("name == 'Muttenz'")

GeoPandas: Geometrie Spalte erstellen mittels Shapely

In [ ]:
from shapely.geometry import Point

geometry = [Point(pos) for pos in zip(df2['lng'], df2['lat'])]
gdf = gpd.GeoDataFrame(df2, geometry=geometry)

In [ ]:
gdf.head()

In [ ]:
gdf.plot()

Exportieren

In [ ]:
gdf.to_file("daten/countries.shp", driver='ESRI Shapefile', encoding="utf-8")

In [ ]:
gdf.to_file("daten/countries.json", driver='GeoJSON', encoding="utf-8")

In [ ]:
gdf.to_file("daten/countries.gpkg", driver='GPKG', encoding="utf-8")

Die Dokumentation zu geopandas ist auf: http://geopandas.readthedocs.io

## Kantone

Wir laden den Datensatz "Generalisierte Gemeindegrenzen" vom Bundesamt für Statistik herunter. Aus Performance-Gründen verwenden wir heute generalisierte Daten. Der detailliertere Datensatz wäre natürlich **swissBOUNDARIES3D** von swisstopo, welcher auch kostenlos bezogen werden kann. ( https://shop.swisstopo.admin.ch/de/products/landscape/boundaries3D )

In [ ]:
import urllib
import shutil

url = "https://www.bfs.admin.ch/bfsstatic/dam/assets/16804410/master"
filename, headers = urllib.request.urlretrieve(url, "daten/gemeindegrenzen.zip")
shutil.unpack_archive("daten/gemeindegrenzen.zip", "daten/gemeindegrenzen")

In [ ]:
kantone = gpd.read_file("daten/gemeindegrenzen/ggg_2021-LV95/shp/g1k21.shp", encoding="utf-8")

#### Überblick des Datensatzes

Sehen wir uns die ersten 5 Einträge an:

In [ ]:
kantone.head(3)

In [ ]:
kantone.head()

In [ ]:
kantone.columns

* Die **Beschreibung** der Attribute ist im PDF des Datensatzes zu finden.

* In diesem Datensatz haben wir sehr viele Sachdaten, welche auch eine Geometrie beschreiben. Das ist eigenlich unschön, sehen wir uns das später an.

In [ ]:
kantone.shape

In [ ]:
kantone.plot(figsize=(15,9));

In [ ]:
zh = kantone.query("KTNAME == 'Zürich'")
zh

In [ ]:
zh.plot()

In [ ]:
kantone.crs

In [ ]:
kantoneWGS84 = kantone.to_crs(epsg=4326)
kantoneWGS84.plot(figsize=(15,9));

In [ ]:
kantoneWeb = kantone.to_crs(epsg=3857)
kantoneWeb.plot(figsize=(15,9));

# Darstellung in Folium

In [ ]:
import folium

center = [47.534018, 7.638423]
karte = folium.Map(center, zoom_start=6, tiles='cartodbpositron')   
# weitere Karten siehe: https://deparkes.co.uk/2016/06/10/folium-map-tiles/

folium.GeoJson(kantoneWGS84).add_to(karte)

karte

In [ ]:
import folium

def my_color_function(feature):
    area = feature["properties"]["AREA_HA"]
    
    color = "#000000"
    
    if area >10000:
        color = "#110000"
    if area >40000:
        color = "#440000"
    if area >70000:
        color = "#77FF00"
    if area >100000:
        color = "#AA0000"
    if area >130000:
        color = "#DDFF00"
    if area >160000:
        color = "#FF0000"
    
    return color

center = [47.534018, 7.638423]
karte = folium.Map(center, zoom_start=6, tiles='cartodbpositron')   
# weitere Karten siehe: https://deparkes.co.uk/2016/06/10/folium-map-tiles/

folium.GeoJson(kantoneWGS84,style_function=lambda feature: {
        'fillColor': my_color_function(feature),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5'
    }).add_to(karte)

karte

## Abfragen

Anlegen eines neuen GeoDataFrames mit reduzierter Anzahl von Attributen (einfach für bessere Übersicht)

In [ ]:
kt = kantone[["KTNR","KTNAME", "AREA_HA", "geometry"]]

In [ ]:
kt.head()

Der Datensatz kann nun gespeichert werden, z.B. als ERSI Shapefile

In [ ]:
kt.to_file("daten/kantone_reduziert.shp", driver="Shapefile", encoding="utf-8")

Laden des Shapefiles (nur so zur Demo)

In [ ]:
kt2 = gpd.read_file("daten/kantone_reduziert.shp", encoding="utf-8")
kt2.head()

In [ ]:
del kt2

GeoPandas enthält zahlreiche Funktionalität, aus Zeitgründen werden wir heute nur ein paar davon ansehen.

Wir sortieren z.B. die Daten nach der Fläche. Weitere Operationen sehen wir mit einem anderen Datensatz an.

In [ ]:
kt.sort_values(['AREA_HA'], ascending=False)

### Räumliche Abfragen / Operationen

In [ ]:
from shapely.geometry import Point

HB = Point(2683022, 1248035)  # Zürich HB (LV95)

In [ ]:
kt.contains(HB)

In [ ]:
kt[kt.contains(HB)]

In [ ]:
kt[kt.contains(HB)].plot()

In [ ]:
dist = kt.distance(HB) / 1000
dist

In [ ]:
kt = kantone[["KTNR","KTNAME", "AREA_HA", "geometry"]].copy()
kt["distance"] = dist
kt

In [ ]:
kt.sort_values(["distance"], ascending=True)

## Visualisierung der Daten

Bar plots, Histograms, Box Plots, Area Plot, Scatter Plot, Hexagonal Bin Plot, Pie plot

Siehe auch: https://pandas.pydata.org/pandas-docs/stable/visualization.html


In [ ]:
kt[["AREA_HA"]].hist();

In [ ]:
kt[["AREA_HA"]].hist(bins=20);

## Nächster Datensatz: Gemeinden

Sehen wir uns einen weiteren, Datensatz von BFS an: Aufteilung in Gemeinden

In [ ]:
gemeinden = gpd.read_file("daten/gemeindegrenzen/ggg_2021-LV95/shp/g1g21_18042021.shp", encoding="utf-8")

In [ ]:
gemeinden.shape

In [ ]:
gemeinden.columns

In [ ]:
gem = gemeinden[["GMDNAME", "GMDNR", "KTNR", "AREA_HA", "geometry"]]
gem.head()

In [ ]:
for i in range(0,len(gem)):
    gs = gem.iloc[i]
    if gs.AREA_HA > 20000:
        print(gs.GMDNAME, gs.AREA_HA)

In [ ]:
gem[gem.AREA_HA > 20000]

In [ ]:
gem[gem.AREA_HA > 20000].plot()

In [ ]:
gem.query("AREA_HA > 20000 and AREA_HA < 30000")

In [ ]:
gem.query("GMDNAME == 'Zürich'")

In [ ]:
gem.query("GMDNAME == 'Muttenz'")

In [ ]:
gem.query("GMDNAME == 'Muttenz' or GMDNAME == 'Pratteln'")

In [ ]:
gem.query("GMDNAME == 'Muttenz' or GMDNAME == 'Pratteln'").plot()

In [ ]:
gem.query("GMDNAME == 'Basel'").plot(figsize=(16,8), color="orange")

Mit Matplotlib mehrere Geometrien plotten:

In [ ]:
kantonsplot = kantone.plot(figsize=(15,9), color="black")

basel = gem.query("GMDNAME == 'Basel'")
zurich = gem.query("GMDNAME == 'Zürich'")

basel.plot(ax=kantonsplot, color="orange")
zurich.plot(ax=kantonsplot, color="blue")


In [ ]:
kantonsplot = kantone.plot(figsize=(15,9), color="gray")

basel = gem.query("GMDNAME == 'Basel'")
allzurich = gem.query("KTNR == 1")

basel.plot(ax=kantonsplot, color="orange", alpha=0.5)
allzurich.plot(ax=kantonsplot, color="blue", alpha=0.25)


## Live Daten vom Internet: Erdbeben

https://earthquake.usgs.gov/earthquakes/feed/v1.0/geojson.php

In [ ]:
import requests

url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_week.geojson"
#url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.geojson"
#url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_month.geojson"

data = requests.get(url)
file = open("earthquakes.geojson","wb")
file.write(data.content)
file.close()

für die Beschreibung der Attribute:

https://earthquake.usgs.gov/data/comcat/data-eventterms.php

In [ ]:
erdbeben = gpd.read_file("earthquakes.geojson")
erdbeben.head()

In [ ]:
erdbeben.shape

In [ ]:
erdbeben.columns

In [ ]:
eb = erdbeben[["time","mag", "place","geometry"]].copy()
eb.head()

In [ ]:
eb.mag.hist(bins=32);

In [ ]:
from datetime import datetime, timezone
datetime.fromtimestamp(1909090919, timezone.utc)

In [ ]:
from datetime import datetime, timezone

data = []
for zeile in range(0,len(eb)):
    time = eb.iloc[zeile].time
    t = str(datetime.fromtimestamp(time/1000.0, timezone.utc))
    data.append(t)
    
eb["time_utc"] = data
eb.head()

### Nach Magnitude (mag) sortieren

Wie ?

In [ ]:
eb.sort_values(["mag"], ascending=False).head()

In [ ]:
eb.plot(markersize=2)

### GeoPandas und PostGIS (Ausblick)

    import geopandas as gpd
    import psycopg2

    con = psycopg2.connect(database="db", user="user",password="pw",host="host")

    sql= "select geom, x,y,z from your_table"

    df=gpd.GeoDataFrame.from_postgis(sql,con,geom_col='geom')